In [3]:
import pandas as pd

# Load all required sheets from the Excel file
def load_data_from_excel(file_path):
    xls = pd.ExcelFile(file_path)
    return {
        "roles": xls.parse(sheet_name=3),
        "matches": xls.parse(sheet_name=0),
        "points": xls.parse(sheet_name="Punti"),
        "parameters": xls.parse(sheet_name="Parametri")
    }

# Rename columns based on the provided translation dictionary
def rename_columns(df, translations):
    return df.rename(columns=translations)

# Derive additional columns for matches
def transform_matches(df):
    df["Winning Team"] = df.apply(
        lambda row: "A" if row["Team A Goals"] > row["Team B Goals"] else ("B" if row["Team A Goals"] < row["Team B Goals"] else "X"), axis=1
    )
    df["Total Goals"] = df["Team A Goals"] + df["Team B Goals"]
    return df[["Date", "Match Type", "Team A Goals", "Team B Goals", "Winning Team", "Total Goals"]]

# Derive additional columns for player points
def transform_points(df):
    df['Team Color'].replace({'Scuri': 'B', 'Bianchi': 'A'}, inplace=True)
    replacement_dict = {'Sì': 1, 'No': 0}
    for column in ['SPL Bonus', 'MVP', 'Penalties']:
        df[column].replace(replacement_dict, inplace=True)
    return df[['Date', 'Player', 'Team Color', 'Goals', 'Own Goals', 'SPL Bonus', 'MVP', 'Friends Brought', 'Penalties']]

# Compute game outcomes and goals scored/conceded for players
def compute_game_details(df):
    df['Game Outcome'] = df.apply(lambda row: 'Won' if row['Team Color'] == row['Winning Team'] else ('Lost' if row['Winning Team'] != 'X' else 'Draw'), axis=1)
    df['Goals Conceded'] = df.apply(lambda row: row['Team B Goals'] if row['Team Color'] == 'A' else row['Team A Goals'], axis=1)
    df['Goals Scored'] = df.apply(lambda row: row['Team A Goals'] if row['Team Color'] == 'A' else row['Team B Goals'], axis=1)
    return df

# Adjust positions for different match types
def adjust_positions(df):
    df['Position 7'] = df['Position'].apply(lambda x: 'Defensive' if x == 'Defender' else ('Offensive' if x in ['Midfielder', 'Forward'] else 'Goalkeeper'))
    df['Position 5'] = df['Position'].apply(lambda x: 'Outfield' if x in ['Defender', 'Midfielder', 'Forward'] else 'Goalkeeper')
    return df[['Player', 'Position 7', 'Position 5']]

# Convert parameters DataFrame to a dictionary for easy lookup
def parameters_to_dict(df):
    return {
        '5-a-side': df.set_index('Parameter')['5-a-side'].to_dict(),
        '7-a-side': df.set_index('Parameter')['7-a-side'].to_dict(),
        '11-a-side': df.set_index('Parameter')['11-a-side'].to_dict()
    }

# Retrieve a parameter value based on match type and parameter name
def get_parameter_value(match_type, parameter, params_dict):
    if match_type <= 10:
        return params_dict['5-a-side'].get(parameter, 0)
    elif match_type <= 16:
        return params_dict['7-a-side'].get(parameter, 0)
    else:
        return params_dict['11-a-side'].get(parameter, 0)

# Compute scores for various categories based on match type and player performance
def compute_scores(df, params_dict):
    df['Participation Points'] = df['Match Type'].apply(lambda x: get_parameter_value(x, 'Participation', params_dict))
    df['Game Outcome Points'] = df.apply(lambda row: get_parameter_value(row['Match Type'], row['Game Outcome']), axis=1)
    df['Goal Points'] = df.apply(lambda row: row['Goals'] * get_parameter_value(row['Match Type'], 'Goal', params_dict), axis=1)
    df['Own Goal Points'] = df.apply(lambda row: row['Own Goals'] * get_parameter_value(row['Match Type'], 'Own Goal', params_dict), axis=1)
    df['Penalty Points'] = df.apply(lambda row: row['Penalties'] * get_parameter_value(row['Match Type'], 'Penalty', params_dict), axis=1)
    df['SPL Bonus Points'] = df.apply(lambda row: row['SPL Bonus'] * get_parameter_value(row['Match Type'], 'SPL Bonus', params_dict), axis=1)
    df['MVP Points'] = df.apply(lambda row: row['MVP'] * get_parameter_value(row['Match Type'], 'MVP', params_dict), axis=1)
    df['Friends Brought Points'] = df.apply(lambda row: row['Friends Brought'] * get_parameter_value(row['Match Type'], 'Friend Brought', params_dict), axis=1)
    return df

data = load_data_from_excel("FantaSPL_V2.2.xlsx")

# Column mappings and translations
translations = {
    "Difensore": "Defender", "Centrocampista": "Midfielder", "Attaccante": "Forward", "Portiere": "Goalkeeper",
    "Data": "Date", "Tipo Partita": "Match Type", "Bianchi": "Team A Goals", "Scuri": "Team B Goals",
    "Squadra vincente": "Winning Team", "Numero Goal": "Number of Goals", "Goal Check": "Goal Check",
    "Numero Goal Punti": "Goal Points", "Autogoal": "Own Goals", "Costo per Partita": "Cost per Match",
    "Effettivo": "Actual Cost"
}

# Data transformations
data['matches'] = rename_columns(data['matches'], translations)
data['matches'] = transform_matches(data['matches'])

data['points'] = rename_columns(data['points'], translations)
data['points'] = transform_points(data['points'])

merged_data = compute_game_details(data['points'].merge(data['matches'], on='Date', how='inner'))

data['roles']['Position'] = data['roles']['Ruolo'].map(translations)
position_data = adjust_positions(data['roles'])

# Merge data and compute scores
merged_data = merged_data.merge(position_data, on='Player', how='left')
params_dict = parameters_to_dict(data['parameters'])
final_data = compute_scores(merged_data, params_dict)

final_data.head(14)


KeyError: 'Team Color'